# Shortwave Radiation Transfer Example

This notebook demonstrates how to compute shortwave radiation fluxes using the `pyrte_rrtmgp` library. Specifically:

- It downloads the required data
- Computes the gas optics using a two-stream approximation
- Solves the radiation transfer equations
- Validates the results against reference datasets

See the [documentation](https://pyrte-rrtmgp.readthedocs.io/en/latest/) for more information.

In [ ]:
import os

import numpy as np
import xarray as xr

## Importing pyrte_rrtmgp Modules

We import the necessary modules from the `pyrte_rrtmgp` package which provide tools for gas optics and solving the radiation transfer equations.

In [ ]:
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver

ERROR_TOLERANCE = 1e-7

## Data Download and Directory Setup

Download the RRTMGP data and define paths for the input and reference directories.

In [ ]:
rte_rrtmgp_dir = download_rrtmgp_data()
rfmip_dir = os.path.join(rte_rrtmgp_dir, "examples", "rfmip-clear-sky")
input_dir = os.path.join(rfmip_dir, "inputs")
ref_dir = os.path.join(rfmip_dir, "reference")
os.listdir(rte_rrtmgp_dir)

## Loading Gas Optics Data and Atmosphere Dataset

Load the shortwave gas optics file and the associated atmosphere dataset that will be used for the simulation.

In [ ]:
gas_optics_sw = load_gas_optics(gas_optics_file=GasOpticsFiles.SW_G224)

atmosphere_file = "multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
atmosphere_path = os.path.join(input_dir, atmosphere_file)
atmosphere = xr.load_dataset(atmosphere_path, chunks={"expt": 3})
atmosphere

## Computing Gas Optics and Solving Radiation Transfer Equations

Compute the shortwave gas optics using a two-stream approximation and then solve the radiation transfer equations to obtain the fluxes.

In [ ]:
gas_optics_sw.gas_optics.compute(atmosphere, problem_type="two-stream")

solver = RTESolver()
fluxes = solver.solve(atmosphere, add_to_input=False)
fluxes

## Loading Reference Data and Validating the Results

Load the reference datasets for upward and downward shortwave fluxes and validate that the computed fluxes match the reference data within the specified tolerance.

In [ ]:
rsu_reference = f"{ref_dir}/rsu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rsd_reference = f"{ref_dir}/rsd_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rsu = xr.load_dataset(rsu_reference, decode_cf=False)
rsd = xr.load_dataset(rsd_reference, decode_cf=False)

assert np.isclose(
    fluxes["sw_flux_up"], rsu["rsu"], atol=ERROR_TOLERANCE
).all(), "Shortwave flux up mismatch"
assert np.isclose(
    fluxes["sw_flux_down"], rsd["rsd"], atol=ERROR_TOLERANCE
).all(), "Shortwave flux down mismatch"

print("Shortwave radiation transfer calculations validated successfully!")